In [1]:
#solving Travelling Sale Person via Ant Cololy Optimization

#coding:utf-8
import os
import numpy as np
import matplotlib.pyplot as pltng
import pandas as pd
from scipy.spatial import distance as dis


In [2]:
print("Working Directory")
print(os.getcwd() )


Working Directory
/Users/machioka_masaki/Downloads/20181001_SatoResult/20180911_SatoSimuTrial004


In [3]:


#Import Picking Order list
PikOrder000 = pd.read_csv("TanaGrid.csv")
PikOrder000

,Tana,x,y
0,B90275,31,34
1,B50524,28,29
2,AL2011,8,21
3,F51031,52,33
4,F60514,52,28
5,G81121,54,10
6,J26621,82,4
7,J35131,84,20
8,JE1021,104,11
9,NJ1711,152,69


In [4]:
#caliculate distance matrix


#Create Visiting Point Logging Matrix

df0mat = ( PikOrder000['x'].astype(str) + '-'  + PikOrder000['y'].astype(str) ).to_numpy() 

df0mat

#Delete duplication
df4 = PikOrder000.drop_duplicates(['x','y']).copy()
df4.reset_index()
df5 = df4[["x","y"]]

locations=np.array(df5)
locs=locations
""" Function defining Position Coordinates """
self_loc = locs                            # x,y axis
self_n_data = len(self_loc)                        # number of data

#Moving distance X axis

self_locX0 = np.copy(self_loc) # Copy Array for caliculate x axis distance
self_locX0[:,1]=0 #Assigning zero for y axis
self_distX0 = dis.cdist(self_locX0,self_locX0,metric='euclidean')    # moving distance x axis



#moving distance Y axis
self_locY0 = np.copy(self_loc) # 
self_locY0[:,0]=0 #
self_distY0 = dis.cdist(self_locY0,self_locY0,metric='euclidean')    # 


##### Total moving disntance equals sum of X axis distance + Y axis distance ####
self_dist0 = self_distX0 + self_distY0

####self_dist_aco --- 
self_dist_aco0 = self_dist0


In [5]:
#The csv to be imported has a row name header on the first row, but does not add a column name to the first column
self_distX1 = pd.read_csv("self_distX0Edited.csv").values
self_distY1 = pd.read_csv("self_distY0Edited.csv").values



In [6]:
###number of iterations
Kaisu = 100



###number of ants
n_agent=1000

#maximum matrix address
maxX = 157 #Kobayashi's Mapß
maxY = 72

In [7]:
#Variable for time name of result output file
import time
timestr = time.strftime("%Y%m%d_%H%M%S")



for i in range( Kaisu ):


    #class TSP:
    #def __init__(self,path=None,alpha = 1.0,beta = 1.0,Q = 1.0,vanish_ratio = 0.95):
    alpha = 1.0
    beta = 1.0
    Q = 1.0
    vanish_ratio = 0.95

    """ Initialization Function """
    self_alpha = alpha                    # Pheromone priority
    self_beta = beta                    # Priority of heuristic information (distance)
    self_Q = Q                            # Coefficient of pheromone variation
    self_vanish_ratio = vanish_ratio    # pheromone evaporation rate

    #locations=np.array(df5)

    #Get distance matrix data
    ##### ####The total distance is the sum of the X-axis distance and the Y-axis distance (after adjusting for column differences)　
    self_dist = self_distX1 + self_distY1
    ####self_dist_aco --- 
    self_dist_aco = self_dist

    #Pheromones and order preservation
    self_weight = np.random.random_sample((self_n_data,self_n_data))    # amount of pheromones
    self_result = np.arange(self_n_data)            # save best order

    


    #def solve(self,n_agent=1000):
    """ Solving the Traveling Salesman Problem with Ant Colony Optimization """

    
    order = np.zeros(self_n_data,np.int64)         # patrol route
    delta = np.zeros((self_n_data,self_n_data))    #Pheromone change amount

    #n_agent=1000
    for k in range(n_agent):
        city = np.arange(self_n_data)
        
        now_city = 0 # Set the starting point x1y1 with the first city number as 0
        #print(now_city)
        last_city = self_n_data - 1

        city = city[ city != now_city ]
        city = city[ city != last_city ]
        order[0] = now_city
        order[last_city] = last_city

        for j in range(1,self_n_data-1): #The range's "self_n_data-1" is because the end point is stored at the end of the range

            upper = np.power(self_weight[now_city,city],self_alpha)*np.power(self_dist_aco[now_city,city],-self_beta)

            evaluation = upper / np.sum(upper)                # Evaluation function
            percentage = evaluation / np.sum(evaluation)    # movement probability
            n_percentage = len(percentage)

            while True:
                index = np.random.randint(n_percentage)
                y = np.random.random()
                if y < percentage[index]:
                    #return index        
                    break
            #print(index)
            # state update
            now_city = city[ index ]
            city = city[ city != now_city ]
            order[j] = now_city


        #L = self_cost(order) # Calculate route cost
        n_order = len(order)
        #L = np.sum( [ self_dist_aco[order[i],order[(i+1)%n_order]] for i in np.arange(n_order) ] )
        L = np.sum( [ (self_dist_aco[order[i],order[(i+1)]])  for i in np.arange(n_order-1) ] )

        # Calculate the amount of change in pheromone
        delta[:,:] = 0.0
        c = self_Q / L
        for j in range(self_n_data-1):
            delta[order[j],order[j+1]] = c
            delta[order[j+1],order[j]] = c

        # Pheromone information update
        self_weight *= self_vanish_ratio 
        self_weight += delta


        #print(self_result)
        # Update result if best ever
        n_order2 = len(self_result)
        #L2 =  np.sum( [ self_dist_aco[self_result[i],self_result[(i+1)%n_order2]] for i in np.arange(n_order2) ] )
        L2 = np.sum( [ (self_dist_aco[self_result[i],self_result[(i+1)]])  for i in np.arange(n_order2-1) ] )

        if L2 > L:
            self_result = order.copy()

    #https://qiita.com/a_t_a/items/50623aedf7e1dab3b876
    import csv
    with open(timestr + '_result.csv','a',newline='') as f:
        writer = csv.writer(f, lineterminator='\n') # Specify line feed code (\n)
        # Header
        #header = ['Route', 'Distance']
        #writer.writerow(header)
        # keep data in list
        csvlist = []
        csvlist.append( df0mat )
        csvlist.append(self_result)
        csvlist.append(  L2  ) #stem distance
        writer.writerow(csvlist)
        f.close()
